# Export


In [26]:
from keras.models import load_model
import utils_io
import numpy as np
import pandas as pd

# Prepare data


TEST DATA


In [27]:
path_test_data = "data_set/test.csv"

test_data = utils_io.load_csv_data(path_test_data)
# Debugging: Check the type of test_data

# Print the loaded data
if isinstance(test_data, pd.DataFrame):
    print(f"dimension train data: {test_data.shape}")
else:
    print("test_data is not a DataFrame")

Successfully loaded data from data_set/test.csv
dimension train data: (1459, 80)


target data


In [28]:
path_target_data = "data_set/sample_submission.csv"

target_data = utils_io.load_csv_data(path_target_data)

# Print the loaded data
if isinstance(target_data, pd.DataFrame):
    print(f"dimension train data: {target_data.shape}")
else:
    print("test_data is not a DataFrame")

Successfully loaded data from data_set/sample_submission.csv
dimension train data: (1459, 2)


In [29]:
# Drop the Id and sales price column
test_data = test_data.drop(columns=["Id"])
print(test_data.shape)

(1459, 79)


Clean numbers NA to na.nan


In [30]:
import pandas as pd
import numpy as np


def auto_clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df_cleaned = df.copy()

    for column in df_cleaned.columns:
        values = df_cleaned[column].dropna().unique()

        # Pokud vše je číslo nebo 'NA' → převedeme na float
        can_be_numeric = True
        for v in values:
            if isinstance(v, str) and v.strip().upper() == "NA":
                continue
            try:
                float(v)
            except:
                can_be_numeric = False
                break

        if can_be_numeric:
            # Převést "NA" na np.nan a sloupec na float
            df_cleaned[column] = df_cleaned[column].replace("NA", np.nan)
            df_cleaned[column] = pd.to_numeric(df_cleaned[column], errors="coerce")
        else:
            # ponecháme jako kategorii se stringy
            df_cleaned[column] = df_cleaned[column].astype(str).str.strip().str.upper()

    return df_cleaned


test_data = auto_clean_columns(test_data)
print(test_data)

      MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0             20       RH         80.0    11622   PAVE    NA      REG   
1             20       RL         81.0    14267   PAVE    NA      IR1   
2             60       RL         74.0    13830   PAVE    NA      IR1   
3             60       RL         78.0     9978   PAVE    NA      IR1   
4            120       RL         43.0     5005   PAVE    NA      IR1   
...          ...      ...          ...      ...    ...   ...      ...   
1454         160       RM         21.0     1936   PAVE    NA      REG   
1455         160       RM         21.0     1894   PAVE    NA      REG   
1456          20       RL        160.0    20000   PAVE    NA      REG   
1457          85       RL         62.0    10441   PAVE    NA      REG   
1458          60       RL         74.0     9627   PAVE    NA      REG   

     LandContour Utilities LotConfig  ... ScreenPorch PoolArea PoolQC  Fence  \
0            LVL    ALLPUB    INSIDE  ...  

## Normalization


In [31]:
import json
import re

json_path = "data_set/generated_description.json"

generated_description = json.load(open(json_path))


for column in test_data.columns:
    for index, value in test_data[column].items():
        # if value == "NA":
        #     print(column, index, value)
        if isinstance(value, str):
            try:
                test_data.at[index, column] = generated_description[column]["items"][
                    value.strip().upper()
                ]["id"]
            except:
                print(column, index, value, isinstance(value, str))

In [32]:
print(test_data.iloc[:10])

   MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0          20        5         80.0    11622      1     0        1   
1          20        1         81.0    14267      1     0        2   
2          60        1         74.0    13830      1     0        2   
3          60        1         78.0     9978      1     0        2   
4         120        1         43.0     5005      1     0        2   
5          60        1         75.0    10000      1     0        2   
6          20        1          NaN     7980      1     0        2   
7          60        1         63.0     8402      1     0        2   
8          20        1         85.0    10176      1     0        1   
9          20        1         70.0     8400      1     0        1   

  LandContour Utilities LotConfig  ... ScreenPorch PoolArea PoolQC Fence  \
0           1         1         1  ...         120        0      0     1   
1           1         1         3  ...           0        0      0     0   
2

In [33]:
mean = test_data.mean(axis=0)
# print(mean)
std = test_data.std(axis=0)
# print(std)

train_data = test_data - mean
train_data = test_data / std

In [34]:
train_data = train_data.astype("float32")
target_data = target_data.astype("float32")

# Import model


In [35]:
model = load_model("house_prices_0.keras")

In [36]:
print(train_data.iloc[0])

MSSubClass          0.467870
MSZoning            6.228839
LotFrontage         3.575125
LotArea             2.345265
Street             15.620609
                    ...     
MiscVal             0.000000
MoSold              2.203912
YrSold           1544.086914
SaleType            1.139949
SaleCondition       1.063230
Name: 0, Length: 79, dtype: float32


In [63]:
i = 0
predictions = model.predict(train_data.iloc[[i]])
target_value = target_data.iloc[i, 1]


diff = int(target_value - predictions[0][0])
pred = int(predictions[0][0])
target = int(target_value)

print(
    f"diff = {format(diff, ',').replace(',', ' ')} USD | "
    f"prediction = {format(pred, ',').replace(',', ' ')} USD | "
    f"target = {format(target, ',').replace(',', ' ')} USD"
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
diff = 26 575 USD | prediction = 142 701 USD | target = 169 277 USD


In [61]:
target_data.iloc[2, 1]

183583.69